# Adiabatic Grover's Search Algorithm in the Presence of Correlated Noise

In this notebook, we compare SchWARMA to a full dynamics simulation performed by Mezze via the adiabatic implementation of Grover's search algorithm [1]. We simulate a faulty Grover evolution given by

$$
H(t)=H_{ad}(t) + H_{err}(t),
$$

where $H_{ad}(t) = \alpha(t) H_0 + [1-\alpha(t)]H_p$ represents the ideal adiabatic Hamiltonian. The dynamics are driven by the annealing schedule $\alpha(t)$ from a system described by the initial Hamiltonian $H_0=I-|+\rangle\langle +|$ to a final (problem) Hamiltonian $H_p=I-|+\rangle\langle +|$. The state $|+\rangle$ denotes the equal superposition state and $|m\rangle$ denotes the marked (target) state. The error Hamiltonian $H_{err}(t)=\sum_i\vec{\beta}_i(t) \vec{\sigma}_i$ represents multi-axis noise characterized by random Gaussian, wide-sense stationary variables $\beta^{\mu}_i(t)$, $\mu=x,y,z$, $i=1,\ldots,n$ for an $n$-qubit system. $\vec{\sigma}_i$ represents the Pauli vector for the $i$th qubit.

The comparison shown in this notebook will center around the optimized Grover evolution, whose annealing schedule is given by

$$
\alpha(t) = \frac{1}{2} - \frac{1}{2\sqrt{N-1}}\tan\left[\left(1-\frac{2 t}{T}\right)\arccos\frac{1}{\sqrt{N}}\right],
$$
where $N=2^n$ is the Hilbert space dimension [2]. The marked state is chosen as the collective state $|0\cdots0\rangle$ for all simulations.

The noise model is chosen to represent collective dephasing, where $\vec{\beta}_i(t)=(0,0,\beta^z(t))$ for all $i$.The noise is assumed to be zero mean and therefore, characterized by its two-point correlation function $C(t-t')=\langle \beta^z(t)\beta^z(t')\rangle$. In particular, we assume the correlations to be Gaussian and fully described by $C(t-t')=\frac{A}{2\sqrt{\pi}\tau_c} e^{-(t-t')^2/4\tau^2_c}$, where $\tau_c$ is the correlation time of the noise.

In [ ]:
import numpy as np
import mezze
import mezze.random.SchWARMA as schwarma
import aqc_implementations as aqc_imps
import sys
import copy
import matplotlib.pyplot as plt
import scipy.linalg as la

## Helper Functions

In [ ]:
# Functions for building multi-qubit operators

def tensor_product(state_list):
    psi = 1
    for state in state_list:
        psi = np.kron(psi, state)
    return psi


def build_sz_list(num_qubits):
    iden = np.array([[1,0],[0,1]])
    sz = np.array([[1,0],[0,-1]])
    op_iden = [iden for i in range(num_qubits)]
    op_list = []
    for j in range(num_qubits):
        sz_j = op_iden.copy()
        sz_j[j] = sz
        op_list.append( tensor_product(sz_j) )
    return op_list

In [ ]:
# Function to run ideal and noise Mezze simulations

def run_mezze_sim(pmd, config_specs, noise_specs, return_sim_only=False):
    # Noise specs
    pmd.noise_hints[0]['amp'] = noise_specs['amp']
    pmd.noise_hints[0]['corr_time'] = noise_specs['corr_time']

    # Set the simulation parameters
    config = mezze.SimulationConfig() # Get the config structure (class)
    #mezze.Noise.seed(1000) # Set the rng seed (doesn't work if parallel is set to true below)
    config.num_steps = config_specs['NN'] # Number of time steps to use in the simulation (larger numbers are more accurate)
    config.time_length = config_specs['T'] # Set the time length of the integration
    config.dt = float(config.time_length) / float(config.num_steps) # Set the time-step size
    config.parallel = config_specs['parallel'] # Set to true for parallel Monte Carlo, False otherwise
    config.num_cpus = int(config_specs['num_procs']) #number of cpus to parallelize over
    config.num_runs = config_specs['num_runs'] # This sets the number of Monte Carlo trials to use to compute the average superoperator
    #config.sample_time(1) # This flag tells the code to report out the superoperator at every time step.
    config.get_unitary = True # Report unitary as opposed to Liouvillian
    config.time_sampling = True # set to true to get the channel at intermediate times
    config.sampling_interval = config_specs['sample_steps'] # records the channel after every n time-steps
    config.realization_sampling = True # set true if you want to record every noise(monte-carlo) realization

    # Controls
    def gt(t):
        return 1.0/2.0-1.0/(2.0*np.sqrt(2.0**Nq-1.0))*np.tan((1.0-2.0*t/config_specs['T'])*np.arccos(1.0/np.sqrt(2.0**Nq)))
    b_ctrl = np.array(list(map(gt, np.linspace(0, config_specs['T'], config_specs['NN']))))
    a_ctrl = 1.0 - b_ctrl
    controls = np.transpose([a_ctrl, b_ctrl])

    # Run Simulation
    ham_func = mezze.HamiltonianFunction(pmd)
    ham = mezze.PrecomputedHamiltonian(pmd, controls, config, ham_func.get_function())
    sim = mezze.Simulation(config, pmd, ham)
    report = sim.run()
    
    # Also return Hamiltonians at each time step
    hh = ham.hamiltonian_matrices
    
    if return_sim_only == False:
        hh = ham.hamiltonian_matrices
        return report, hh
    else:
        return sim

In [ ]:
# Function to run SchWARMA simulation of adiabatic dynamics

def schwarma_sim(ideal_evolution_ops, noise_op, noise_traj):
    U_list = []
    U = np.identity(len(noise_op))
    for i in range(len(ideal_evolution_ops)):
        U_ad = ideal_evolution_ops[i]
        U_noise = la.expm(-1.0j*noise_traj[i]*noise_op)
        U = U_noise @ U_ad @ U
        U_list.append(U)
    return np.array(U_list)

In [ ]:
# Functions for calculating statistics of the performance metric

def jackknife_mean(data):
    mean_data = []
    for i in range(len(data)):
        temp_data = list(data[:i]) + list(data[(i+1):])
        mean_data.append(np.mean(temp_data))
    return np.mean(mean_data)

def jackknife_mean_CI(data):
    mean_data = []
    for i in range(len(data)):
        temp_data = list(data[:i]) + list(data[(i+1):])
        mean_data.append(np.mean(temp_data))
    return np.quantile(mean_data, 0.0257), np.quantile(mean_data, 0.975)

## SchWARMA vs. Mezze: Single Instance

This section conveys the efficacy of SchWARMA by examining Grover's search dynamics for $0\leq t \leq T$. SchWARMA dynamics are compared against a full dynamics simulation produced by the Mezze simulation toolbox. The metric of choice is the energy difference between $\epsilon_0(t)$, the time-dependent ground state energy of the ideal Hamiltonian $H_{ad}(t)$, and the average energy of the system $\langle E(t)\rangle = \text{Tr}[\rho(t) H(t)]$, where $\rho(t)=U(t)\rho(0)U^\dagger(t)$ is the state of the system after being evolved by the faulty evolution $U(t)=\exp\left[-i\int^{t}_0 dt' H(t')\right]$.

### Mezze Simulation

The Mezze simulator propagates the faulty dynamics via

$$
U(t_j, t_{j-1}) = \exp\left(-i\int^{t_j}_{t_{j-1}} H(t)dt\right)\approx\prod^{N_j}_{k=1}e^{-iH(k\delta t)\delta t},
$$

where Trotterization is used to approximate the dynamics. The number of Trotter steps is given by $N_j=T/\delta t$, with $\delta t$ denoting the time resolution of the approximate evolution.

In [ ]:
'''
WARNING: This simulation can be time consuming, especially if not performed using multiprocessing
'''

# System specifications
Nq = 2

# Operators
su = np.array([[1],[0]])
sd = np.array([[0],[1]])
sp = 1.0/np.sqrt(2)*(su+sd)
init_state = tensor_product([sp for i in range(Nq)])
marked_state = tensor_product([su for i in range(Nq)])
sz_list = build_sz_list(Nq)
Sz = np.zeros((2**Nq, 2**Nq))
for sz_j in sz_list:
    Sz += sz_j

# Initial State
rho_0 = init_state @ np.conjugate(np.transpose(init_state))
rho_target = marked_state @ np.conjugate(np.transpose(marked_state))

# System Description
pmd = aqc_imps.DephasingGrover(Nq, init_state, marked_state, Sz)

# Noise Specifications (In terms of the min energy gap)
gap = 1/2**(Nq/2)
noise_power = 0.01*gap**2
noise_corr = 5.0*1/gap
noise_amp = noise_power*noise_corr*2*np.sqrt(np.pi) # defined by mezze PSD

# Time and Step-size Specifications
t_gap = 1/gap
dt = 1/8*t_gap
dN = 100 # Number of time steps per dt
num_realz = 100 # Number of noise realizations
T = 4*t_gap # Total simulation time in units of the gap
NN = int(T//dt)*dN

# Simulation Configurations
config_specs = {'num_runs': num_realz, 'sample_steps': 10, 'NN': NN,
                'T': T, 'parallel': False, 'num_procs': 1}
ideal_config_specs = copy.deepcopy(config_specs)
ideal_config_specs['num_runs'] = 1
ideal_config_specs['parallel'] = False

# Noise Configurations
noise_specs = {'amp': noise_amp, 'corr_time': noise_corr}
ideal_noise_specs = {'amp': 0, 'corr_time': 1}


# Run Ideal Evolution
_, h_list = run_mezze_sim(pmd, ideal_config_specs, ideal_noise_specs)

# Run Faulty Evolution
report_faulty, _ = run_mezze_sim(pmd, config_specs, noise_specs)

# Calculate Energy Difference
snapshots = int(config_specs['NN']//config_specs['sample_steps'])
p0_realz = np.zeros([num_realz, snapshots])
for i in range(num_realz):
    print(i)
    for snap_idx in range(snapshots):
        # Calculate Ground State Energy for 0<t<T
        H = h_list[snap_idx*config_specs['sample_steps']]
        Evals, Evecs = np.linalg.eigh(H)
        P0 = np.conjugate(np.transpose([Evecs[0]])) @ [Evecs[0]]
        Ut = report_faulty.time_samples[i][snap_idx].liouvillian()
        rho_t = Ut @ rho_0 @ np.conjugate(np.transpose(Ut))
        E0_avg = abs(Evals[0] - np.trace(rho_t @ H))
        p0_realz[i][snap_idx] = np.real(E0_avg)

In [ ]:
p0_mezze_data = [(jackknife_mean(p0_realz[:,i]), jackknife_mean_CI(p0_realz[:,i])) for i in range(snapshots)]
p0_mezze_data = np.array(p0_mezze_data)

In [ ]:
t_list = np.linspace(0,1,snapshots)
fig = plt.figure(dpi=80)
ax = fig.add_subplot(111)
ax.plot(t_list, p0_mezze_data[:,0],'--')

CI_data = p0_mezze_data[:,1]
p0_top = np.array([CI_data[i][1] for i in range(len(CI_data))])
p0_bottom = np.array([CI_data[i][0] for i in range(len(CI_data))])
ax.fill_between(t_list, p0_top, p0_bottom, alpha=0.3)

ax.set_xlabel('Normalized Time, $t/T$', fontsize=15)
ax.set_ylabel(r'$|\epsilon_0(t)-\langle E(t)\rangle|$', fontsize=15);

### SchWARMA Simulation

We effectively simulate $H_{err}(t)$ and the faulty Grover dynamics by introducing a SchWARMA model into the Trotterized evolution as

$$
    U(T)=\exp\left(-i\int^T_0 H(t)dt\right) \approx \prod^{N}_{j=1} U_E(t_j)U_{ad}(t_j, t_{j-1}),
$$

where $U_E(t_j)=\exp(-i \sum_k y^k_{j} S_k)$. The variables $y^k_{j}$ are defined by the underlying SchWARMA model and thus, they encapsulate the temporal properties of the noise. The evolution

$$
U_{ad}(t_j, t_{j-1}) = \exp\left(-i\int^{t_j}_{t_{j-1}} H_{ad}(t)dt\right)\approx\prod^{N_j}_{k=1}e^{-iH_{ad}(k\delta t)\delta t}
$$

represents the pure adiabatic dynamics that is Trotterized a number of steps $N_j=T/\delta t$, where $\delta t$ is the time resolution of the approximate evolution. Simulating the dynamics in this manner allows one to leverage the noiseless approximation of the adiabatic dynamics as a base for the faulty simulations.

In [ ]:
# System specifications
Nq = 2

# Operators
su = np.array([[1],[0]])
sd = np.array([[0],[1]])
sp = 1.0/np.sqrt(2)*(su+sd)
init_state = tensor_product([sp for i in range(Nq)])
marked_state = tensor_product([su for i in range(Nq)])
sz_list = build_sz_list(Nq)
Sz = np.zeros((2**Nq, 2**Nq))
for sz_j in sz_list:
    Sz += sz_j
noise_op = Sz

# Initial State
rho_0 = init_state @ np.conjugate(np.transpose(init_state))
rho_target = marked_state @ np.conjugate(np.transpose(marked_state))

# System Description
pmd = aqc_imps.DephasingGrover(Nq, init_state, marked_state, Sz)

# Noise Specifications (In terms of the min energy gap)
gap = 1/2**(Nq/2)
noise_power = 0.01*gap**2
noise_corr = 5.0*1/gap
noise_amp = noise_power*noise_corr*2*np.sqrt(np.pi) # defined by mezze PSD

# Time and Step-size Specifications
t_gap = 1/gap # Relevant time scale
t_sample = 1/8*t_gap # Mezze sampling time in terms of min gap
dN = 100 # Number of time steps per t_sample (effective number of gate time steps)
num_realz = 500 # Number of noise realizations
T = 4*t_gap # Total simulation time in units of the gap
NN = int(T//t_sample)*dN # Number of Mezze time steps
dt = T/NN # Mezze time resolution
schwarma_every_n = int(dN) # Number of time steps before noise is added
NN_schwarma = int(NN//schwarma_every_n) # Number of SchWARMA steps (Mezze snapshots)
t_gate = schwarma_every_n*dt # SchWARMA time resolution

# Ideal Simulation Configuration
schwarma_config_specs = {'num_runs': 1, 'sample_steps': schwarma_every_n, 
                'NN': NN, 'T': T, 'parallel': False, 'num_procs': 1}
schwarma_noise_specs = {'amp': 0, 'corr_time': 1}
spectrum_config_specs = {'num_runs': 1, 'sample_steps': 10, 'parallel': False, 'num_procs': 1} # Used only to extract noise spectrum
spectrum_config_specs['T'] = 10*T
spectrum_config_specs['NN'] = NN

# Run Ideal Evolution
report, h_list = run_mezze_sim(pmd, schwarma_config_specs, schwarma_noise_specs)
U_ideal_list = []
U_prev = np.identity(len(noise_op))
for i in range(NN_schwarma):
    U_k = report.time_samples[0][i].liouvillian()
    U_dt = U_k @ (U_prev.conj().T)
    U_prev = U_k
    U_ideal_list.append(U_dt)
    
# Noise Configuration
a = [1,]
spectrum_noise_specs = {'amp': noise_amp, 'corr_time': noise_corr}
mezze_sim = run_mezze_sim(pmd, spectrum_config_specs, spectrum_noise_specs, return_sim_only=True)
b = schwarma.extract_and_downconvert_from_sim(mezze_sim, t_gate)[0]

# Run Faulty Evolution and Calculate Energy Difference
p0_realz = np.zeros([num_realz, NN_schwarma])
for i in range(num_realz):
    print(i)
    schwarma_angles = schwarma.schwarma_trajectory(NN_schwarma, a, b, amp=1)
    U_faulty_list = schwarma_sim(U_ideal_list, noise_op, schwarma_angles)
    
    # Calculate GS Overlap for 0<t<T: |<psi(t)|0(t)>|^2
    for snap_idx in range(NN_schwarma):
        H = h_list[snap_idx*schwarma_config_specs['sample_steps']]
        Evals, Evecs = np.linalg.eigh(H)
        P0 = np.conjugate(np.transpose([Evecs[0]])) @ [Evecs[0]]
        Ut = U_faulty_list[snap_idx]
        rho_t = Ut @ rho_0 @ np.conjugate(np.transpose(Ut))
        E0_avg = abs(Evals[0] - np.trace(rho_t @ H))
        p0_realz[i][snap_idx] = np.real(E0_avg)

In [ ]:
p0_schwarma_data = [(jackknife_mean(p0_realz[:,i]), jackknife_mean_CI(p0_realz[:,i])) for i in range(NN_schwarma)]
p0_schwarma_data = np.array(p0_schwarma_data)

In [ ]:
fig = plt.figure(dpi=80)
ax = fig.add_subplot(111)

# Mezze Data
t_list = np.linspace(0,1,snapshots)
ax.plot(t_list, p0_mezze_data[:,0],'--b', label='Mezze')
CI_data = p0_mezze_data[:,1]
p0_top = np.array([CI_data[i][1] for i in range(len(CI_data))])
p0_bottom = np.array([CI_data[i][0] for i in range(len(CI_data))])
ax.fill_between(t_list, p0_top, p0_bottom, color='b', alpha=0.3)

# SchWARMA Data
t_list = np.linspace(0,1,NN_schwarma)
ax.plot(t_list, p0_schwarma_data[:,0],'--', color='orange', label='SchWARMA')
CI_data = p0_schwarma_data[:,1]
p0_top = np.array([CI_data[i][1] for i in range(len(CI_data))])
p0_bottom = np.array([CI_data[i][0] for i in range(len(CI_data))])
ax.fill_between(t_list, p0_top, p0_bottom, color='orange', alpha=0.3)

ax.set_xlabel('Normalized Time, $t/T$', fontsize=15)
ax.set_ylabel(r'$|\epsilon_0(t)-\langle E(t)\rangle|$', fontsize=15)
ax.legend(loc=4, fontsize=12)

## SchWARMA vs. Mezze: Fidelity vs. Correlation Time

In this section, we present code for reproducing the correlation time comparison shown in the appendix of the SchWARMA manuscript. We fix the total evolution time and vary the correlation time in units of the minimum energy gap between the ground state and first excited state of the ideal adiabatic Hamiltonian. We calculate fidelity via $F=\langle \Phi_0(T) |\rho(T)|\Phi_0(T)\rangle$, where $|\Phi_0(t)\rangle$ is the time-dependent eigenstate of the pure adiabatic Hamiltonian, i.e. $H_{ad}(t)|\Phi_0(t)\rangle=\epsilon_0(t)|\Phi_0(t)\rangle$. The time-evolved state $\rho(t)=U(t)\rho(0)U^{\dagger}(t)$ is calculate via Mezze and SchWARMA.

### Mezze Simulation

In [ ]:
'''
WARNING: This simulation can be time consuming, especially if not performed using multiprocessing
'''

# System specifications
Nq = 2

# Operators
su = np.array([[1],[0]])
sd = np.array([[0],[1]])
sp = 1.0/np.sqrt(2)*(su+sd)
init_state = tensor_product([sp for i in range(Nq)])
marked_state = tensor_product([su for i in range(Nq)])
sz_list = build_sz_list(Nq)
Sz = np.zeros((2**Nq, 2**Nq))
for sz_j in sz_list:
    Sz += sz_j

# Initial State
rho_0 = init_state @ np.conjugate(np.transpose(init_state))
rho_target = marked_state @ np.conjugate(np.transpose(marked_state))

# System Description
pmd = aqc_imps.DephasingGrover(Nq, init_state, marked_state, Sz)

# Noise Specifications (In terms of the min energy gap)
gap = 1/2**(Nq/2)
noise_power = 0.001*gap**2

# Time and Step-size Specifications
t_gap = 1/gap
dt = 1/4*t_gap
dN = 100 # Number of time steps per dt
num_realz = 100 # Number of noise realizations (Increase to 1000 for improved results)
T = 10*t_gap # Total simulation time in units of the gap
NN = int(T//dt)*dN

# Simulation Configurations
config_specs = {'num_runs': num_realz, 'sample_steps': 10, 'NN': NN,
                'T': T, 'parallel': False, 'num_procs': 1}

# Main Simulation: Varying Correlation Time
t_corr_min = 0.1
t_corr_max = 10.0
noise_corr_list = np.logspace(np.log10(t_corr_min), 
                              np.log10(t_corr_max), 150)[::10]*1/gap # remove skip by 10 for manuscript plot
mezze_corr_data = []
for noise_corr in noise_corr_list:
    print(r'tau_c = %2.4f' % noise_corr)
    noise_amp = noise_power*noise_corr*2*np.sqrt(np.pi) # defined by mezze PSD
    
    # Noise Configurations
    noise_specs = {'amp': noise_amp, 'corr_time': noise_corr}

    # Run Faulty Simulation
    report_faulty, _ = run_mezze_sim(pmd, config_specs, noise_specs)
    
    # Calculate Average Fidelity
    F_realz = []
    for i in range(num_realz):
        Ut = report_faulty.time_samples[i][-1].liouvillian()
        rho_t = Ut @ rho_0 @ np.conjugate(np.transpose(Ut))
        F = np.trace(rho_t @ rho_target)
        F_realz.append(F)
    
    # Compute Statistics
    F_med = jackknife_mean(F_realz)
    F_ci = jackknife_mean_CI(F_realz)
    mezze_corr_data.append( [noise_corr*gap, F_med, F_ci] )

In [ ]:
mezze_corr_data = np.array(mezze_corr_data)
fig = plt.figure(dpi=70)
ax = fig.add_subplot(111)
ax.plot(mezze_corr_data[:,0], mezze_corr_data[:,1])

CI_data = mezze_corr_data[:,2]
p0_top = np.array([CI_data[i][1] for i in range(len(CI_data))])
p0_bottom = np.array([CI_data[i][0] for i in range(len(CI_data))])
ax.fill_between(list(mezze_corr_data[:,0]), p0_top, p0_bottom, alpha=0.3)

ax.set_xlabel(r'Correlation Time, $\tau_c$ ($1/\Delta_{min}$)', fontsize=15)
ax.set_ylabel(r'Fidelity', fontsize=15);
ax.set_xscale('log')

### SchWARMA Simulation

In [ ]:
'''
WARNING: This simulation can be time consuming, especially if not performed using multiprocessing
'''

# System specifications
Nq = 2

# Operators
su = np.array([[1],[0]])
sd = np.array([[0],[1]])
sp = 1.0/np.sqrt(2)*(su+sd)
init_state = tensor_product([sp for i in range(Nq)])
marked_state = tensor_product([su for i in range(Nq)])
sz_list = build_sz_list(Nq)
Sz = np.zeros((2**Nq, 2**Nq))
for sz_j in sz_list:
    Sz += sz_j
noise_op = Sz

# Initial State
rho_0 = init_state @ np.conjugate(np.transpose(init_state))
rho_target = marked_state @ np.conjugate(np.transpose(marked_state))

# System Description
pmd = aqc_imps.DephasingGrover(Nq, init_state, marked_state, Sz)

# Noise Specifications (In terms of the min energy gap)
gap = 1/2**(Nq/2)
noise_power = 0.001*gap**2

# Time and Step-size Specifications
t_gap = 1/gap # Relevant time scale
t_sample = 1/4*t_gap # Mezze sampling time in terms of min gap
dN = 100 # Number of time steps per t_sample (effective number of gate time steps)
num_realz = 200 # Number of noise realizations (Increase to 1000 for improved results)
T = 10*t_gap # Total simulation time in units of the gap
NN = int(T//t_sample)*dN # Number of Mezze time steps
dt = T/NN # Mezze time resolution
schwarma_every_n = int(dN) # Number of time steps before noise is added
NN_schwarma = int(NN//schwarma_every_n) # Number of SchWARMA steps (Mezze snapshots)
t_gate = schwarma_every_n*dt # SchWARMA time resolution

# Ideal Simulation Configuration
schwarma_config_specs = {'num_runs': 1, 'sample_steps': schwarma_every_n, 
                'NN': NN, 'T': T, 'parallel': False, 'num_procs': 1}
schwarma_noise_specs = {'amp': 0, 'corr_time': 1}
spectrum_config_specs = {'num_runs': 1, 'sample_steps': 10, 'parallel': False, 'num_procs': 1} # Used only to extract noise spectrum
spectrum_config_specs['T'] = 30*T
spectrum_config_specs['NN'] = int(spectrum_config_specs['T']/t_sample)*dN

# Run Ideal Evolution
report, h_list = run_mezze_sim(pmd, schwarma_config_specs, schwarma_noise_specs)
U_ideal_list = []
U_prev = np.identity(len(noise_op))
for i in range(NN_schwarma):
    U_k = report.time_samples[0][i].liouvillian()
    U_dt = U_k @ (U_prev.conj().T)
    U_prev = U_k
    U_ideal_list.append(U_dt)

# Main Simulation: Varying Correlation Time
t_corr_min = 0.1
t_corr_max = 10.0
noise_corr_list = np.logspace(np.log10(t_corr_min), 
                              np.log10(t_corr_max), 150)[::10]*1/gap
schwarma_corr_data = []
for noise_corr in noise_corr_list:
    print(r'tau_c = %2.4f' % noise_corr)
    # Noise Configuration
    a = [1,]
    noise_amp = noise_power*noise_corr*2*np.sqrt(np.pi) # defined by mezze PSD
    spectrum_noise_specs = {'amp': noise_amp, 'corr_time': noise_corr}
    mezze_sim = run_mezze_sim(pmd, spectrum_config_specs, spectrum_noise_specs, return_sim_only=True)
    b = schwarma.extract_and_downconvert_from_sim(mezze_sim, t_gate)[0]
    # SchWARMA Faulty Evolution
    F_realz = []
    for i in range(num_realz):
        schwarma_angles = schwarma.schwarma_trajectory(NN_schwarma, a, b, amp=1)
        U_faulty_list = schwarma_sim(U_ideal_list, noise_op, schwarma_angles)
        Ut = U_faulty_list[-1]
        rho_t = Ut @ rho_0 @ np.conjugate(np.transpose(Ut))
        F = np.trace(rho_t @ rho_target)
        F_realz.append(np.real(F))
    #F_med = jackknife_median(F_realz)
    #F_ci = jackknife_median_CI(F_realz)
    F_med = jackknife_mean(F_realz)
    F_ci = jackknife_mean_CI(F_realz)
    schwarma_corr_data.append( [noise_corr*gap, F_med, F_ci] )
schwarma_corr_data = np.array(schwarma_corr_data)

In [ ]:
fig = plt.figure(dpi=70)
ax = fig.add_subplot(111)

# SchWARMA Data
ax.plot(schwarma_corr_data[:,0], schwarma_corr_data[:,1],'--', label='SchWARMA')
CI_data = schwarma_corr_data[:,2]
p0_top = np.array([CI_data[i][1] for i in range(len(CI_data))])
p0_bottom = np.array([CI_data[i][0] for i in range(len(CI_data))])
ax.fill_between(list(schwarma_corr_data[:,0]), p0_top, p0_bottom, alpha=0.3, color='b')

# Mezze Data
mezze_corr_data = np.array(mezze_corr_data)
ax.plot(mezze_corr_data[:,0], mezze_corr_data[:,1], 'orange', linestyle='--', label='Mezze')
CI_data = mezze_corr_data[:,2]
p0_top = np.array([CI_data[i][1] for i in range(len(CI_data))])
p0_bottom = np.array([CI_data[i][0] for i in range(len(CI_data))])
ax.fill_between(list(mezze_corr_data[:,0]), p0_top, p0_bottom, alpha=0.3, color='orange')

ax.set_xlabel(r'Correlation Time, $\tau_c$ ($1/\Delta_{min}$)', fontsize=15)
ax.set_ylabel(r'Fidelity', fontsize=15);
ax.set_xscale('log')
ax.set_ylim([0.97,1])
ax.legend(loc=3, fontsize=15)

Increasing the number of noise realizations to 1000 produces the results shown below and in the SchWARMA manuscript.

In [ ]:
import pickle as pk
Nq = 2
s_pow = 0.001
NN = 4000
NNsch = 100

fig = plt.figure(dpi=70)
ax = fig.add_subplot(111)

directory = './data/'
# SchWARMA
tau_c = 7.57
fname = 'grover_dephasing_schwarma_RC_corr_Nq%d_NN%d_NNsch%d_Sw%1.4f_NC%1.2f-v0.p' % (Nq, NN, NNsch, s_pow, tau_c)
sch_fid2 = pk.load(open(directory + fname, 'rb'))
ax.plot(sch_fid2[:,0], sch_fid2[:,1], 'o', color = '#fc5a50', markerfacecolor='None', label=r'SchWARMA')

# Mezze
tau_c = 10.0
fname = 'grover_dephasing_corr_RC_Nq%d_NN%d_Sw%1.4f_NC%1.2f.p' % (Nq, NN, s_pow, tau_c)
mezze_fid2 = pk.load(open(directory + fname, 'rb'))
ax.plot(mezze_fid2[:,0], mezze_fid2[:,1], linestyle='-', color = '#00035b', label='Mezze')

ax.set_xlabel(r'Correlation Time, $\tau_c$ ($1/\Delta_{min}$)', fontsize=15)
ax.set_ylabel(r'Fidelity', fontsize=15);
ax.set_xscale('log')
ax.set_ylim([0.97,1])
ax.legend(loc=3, fontsize=15)

## References

[1] J. Roland and N. Cerf, Phys. Rev. A 65, 042308 (2002)

[2] A.T. Rezakhani, W.-J. Kuo, A. Hamma, D.A. Lidar, and P. Zanardi, Phys. Rev. Lett. 103, 080502 (2009)